In [255]:
import matplotlib.pyplot as plt
import readsnapGadget2 as snap
import numpy as np
from copy import copy

In [256]:
snapnum = 95
filename = f"data/snapshot_{str(snapnum).zfill(3)}"

head = snap.snapshot_header(filename)
redshift = head.redshift

# list of particle positions
pos = snap.read_block(filename, "POS ")

# create copy of particle locations, and create list of indices of particles
particles = pos.copy()
part_ind = np.indices([len(particles)])[0]

# list of particle masses
mass = snap.read_block(filename, "MASS")

# linking length for friends of friends algo
link_len = .2  * 1 #kpc

In [257]:
# dictionary to hold groups
groups = {}

# initial group id to label groups
group_id = 0

# loop while there are still particles to be grouped
while len(part_ind) > 0:
    
    # get first particle index 
    index = part_ind[0]
    print(f'Considering friends of partyicle {index} for friend group {group_id}' )
    
    # get location of first particle 
    
    part = particles[index]
    
    # drop first particle from particle list
    part_ind = np.delete(part_ind, [0])
    
    # calculate distance to particle of interest for all other particles 
    dists = (np.sum(np.abs(particles - part_friend), axis=1))**2
    
    # determine which are within the linking length
    friend_ind = list(set(np.where(dists < link_len)[0]) & set(part_ind))  
    
    print(f'Found friends {friend_ind} for friend group {group_id}\n')
    # add these indices to group 
    groups[group_id] = friend_ind
    
    friend_ind_loop = friend_ind.copy()
    
    # now loop over friends to find more friends!
    
    while len(friend_ind_loop) > 0:
        
        print(f'Looping over friends {friend_ind_loop} for group {group_id}\n')
        
        # consider the first friend
        index_friend = friend_ind_loop[0]
        
        
        friend_ind_loop = np.delete(friend_ind_loop, np.where(friend_ind_loop == index_friend))
        part_ind = np.delete(part_ind, np.where(part_ind == index_friend))
        
        # get position of first friend
        part_friend = particles[index_friend]
        
        # repeat above analysis for first friend 
        dists = (np.sum(np.abs(particles - part_friend), axis=1))**2
       
        # determine which are within the linking length only for particles that haven't been considered
        friend_ind = list(set(np.where(dists < link_len)[0]) & set(part_ind)) 
        # if we found more friends,
        if len(friend_ind) > 0:
            
            print(f'Found more friends {friend_ind} of friend {index_friend}, adding to group {group_id}!\n')
            # add to the friend group
            groups[group_id] = np.unique(np.concatenate([groups[group_id], friend_ind]))
            
            # add new friends to loop
            friend_ind_loop = np.unique(np.concatenate([friend_ind_loop, friend_ind]))
            
            print
        # if there are no more friends , friend group is complete
        elif len(friend_ind_loop) == 0:
            print(f'no more friends to consider, friend group {group_id} is complete\n')
            group_id +=1
            
            
            

Considering friends of partyicle 0 for friend group 0
Found friends [3979, 3980] for friend group 0

Looping over friends [3979, 3980] for group 0

Found more friends [3975, 3976, 3977, 3978, 3980, 3921, 3922, 3924, 3925, 3926, 3927, 3928, 3929, 3930, 3931, 3932, 3933, 3934, 3935, 3937, 3938, 3939, 3940, 3941, 3942, 3943, 3944, 3945, 3946, 3947, 3948, 3949, 3950, 3951, 3952, 3953, 3954, 3955, 3956, 3957, 3958, 3959, 3705, 3706] of friend 3979, adding to group 0!

Looping over friends [3705 3706 3921 3922 3924 3925 3926 3927 3928 3929 3930 3931 3932 3933
 3934 3935 3937 3938 3939 3940 3941 3942 3943 3944 3945 3946 3947 3948
 3949 3950 3951 3952 3953 3954 3955 3956 3957 3958 3959 3975 3976 3977
 3978 3980] for group 0

Found more friends [3975, 3976, 3977, 3978, 3727, 3728, 3926, 3927, 3928, 3929, 3930, 3931, 3932, 3933, 3934, 3935, 3936, 3944, 3949, 3950, 3952, 3701, 3702, 3703, 3704, 3706] of friend 3705, adding to group 0!

Looping over friends [3701 3702 3703 3704 3706 3727 3728 3921

KeyboardInterrupt: 

[2, 3]